## Work
1. 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現

In [1]:
import os
import keras

# 本作業可以不需使用 GPU, 將 GPU 設定為 "無" (若想使用可自行開啟)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
del train
del test

In [6]:

import tensorflow as tf
# 只使用 30% 的 GPU 記憶體
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# 設定 Keras 使用的 TensorFlow Session
tf.keras.backend.set_session(sess)


In [7]:
def build_mlp(input_shape, output_units=10, num_neurons=[256, 128, 64]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [8]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 0.005
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = [0.95, 0.85, 0.75]

In [9]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for M in MOMENTUM:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with MOMENTUM = %.6f" % (M))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=M)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-MOMENTUM-%s" % str(M)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


Experiment with MOMENTUM = 0.950000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
hidden_layer2 (Dense)        (None, 128)               32896     
_________________________________________________________________
hidden_layer3 (Dense)        (None, 64)                8256      
_________________________________________________________________
output (Dense)               (None, 10)                650       
Total params: 828,490
Trainable params: 828,490
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==========================

Epoch 1/50
50000/50000 [==============================] - 3s 55us/step - loss: 1.9721 - acc: 0.2918 - val_loss: 1.8895 - val_acc: 0.3329
Epoch 2/50
50000/50000 [==============================] - 2s 46us/step - loss: 1.7680 - acc: 0.3728 - val_loss: 1.7730 - val_acc: 0.3656
Epoch 3/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.6851 - acc: 0.4064 - val_loss: 1.7598 - val_acc: 0.3842
Epoch 4/50
50000/50000 [==============================] - 2s 49us/step - loss: 1.6282 - acc: 0.4272 - val_loss: 1.6404 - val_acc: 0.4193
Epoch 5/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.5799 - acc: 0.4436 - val_loss: 1.5926 - val_acc: 0.4306
Epoch 6/50
50000/50000 [==============================] - 2s 45us/step - loss: 1.5463 - acc: 0.4549 - val_loss: 1.5537 - val_acc: 0.4478
Epoch 7/50
50000/50000 [==============================] - 2s 46us/step - loss: 1.5118 - acc: 0.4673 - val_loss: 1.5564 - val_acc: 0.4386
Epoch 8/50
50000/50000 [=================

50000/50000 [==============================] - 2s 44us/step - loss: 1.8359 - acc: 0.3513 - val_loss: 1.8199 - val_acc: 0.3562
Epoch 3/50
50000/50000 [==============================] - 3s 50us/step - loss: 1.7630 - acc: 0.3794 - val_loss: 1.7301 - val_acc: 0.3923
Epoch 4/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.7070 - acc: 0.3990 - val_loss: 1.7372 - val_acc: 0.3816
Epoch 5/50
50000/50000 [==============================] - 2s 47us/step - loss: 1.6670 - acc: 0.4109 - val_loss: 1.7191 - val_acc: 0.3905
Epoch 6/50
50000/50000 [==============================] - 2s 45us/step - loss: 1.6312 - acc: 0.4251 - val_loss: 1.6322 - val_acc: 0.4213
Epoch 7/50
50000/50000 [==============================] - 2s 44us/step - loss: 1.6019 - acc: 0.4355 - val_loss: 1.6244 - val_acc: 0.4177
Epoch 8/50
50000/50000 [==============================] - 2s 45us/step - loss: 1.5724 - acc: 0.4460 - val_loss: 1.6139 - val_acc: 0.4298
Epoch 9/50
50000/50000 [============================

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ["r", "g", "b", "k", "y"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()